# Download and import librairies

In [1]:
# Import Librairies
!pip install gspread_dataframe gspread oauth2client
!pip install gspread-pandas
!{sys.executable} -m pip install pygsheets
from gspread_pandas import Spread, Client
from google.oauth2.service_account import Credentials
import pandas as pd
import re
import sys
import json
import gspread
import pygsheets
import csv
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text
from sklearn.cluster import DBSCAN
# import nltk
# from nltk.stem.snowball import SnowballStemmer
# from nltk.stem import PorterStemmer

'{sys.executable}' is not recognized as an internal or external command,
operable program or batch file.


# Import our Google Search Console data from a CSV file

In [2]:
# Import our Google Search Console CSV Data into a Dataframe
df = pd.read_csv('Queries.csv', delimiter=',')
df.head(20)


,Top queries,Clicks,Impressions,CTR,Position
0,adelphi living,289,834,34.65%,1.07
1,adelphi,15,298,5.03%,9.97
2,sil providers perth,12,90,13.33%,3.26
3,supported independent living perth,7,323,2.17%,10.77
4,adelphi living mandurah,7,30,23.33%,1.03
5,disability accommodation perth,3,641,0.47%,31.52
6,ndis providers near me,3,45,6.67%,1.00
7,sda providers perth,3,23,13.04%,7.48
8,disability housing perth,2,541,0.37%,31.37
9,ndis providers perth,2,258,0.78%,52.90


## Create a list of keywords from our GSC data

In [3]:
# Define a variable to hold our 'Top queries' column only
top_queries = df['Top queries']

# Create a list of keywords and populate it from our dataframe
keywords = []

for x in top_queries:
  keywords.append(x)

# Display keywords list
keywords

['adelphi living',
 'adelphi',
 'sil providers perth',
 'supported independent living perth',
 'adelphi living mandurah',
 'disability accommodation perth',
 'ndis providers near me',
 'sda providers perth',
 'disability housing perth',
 'ndis providers perth',
 'sil ndis',
 'ndis housing perth',
 'sil',
 'disability supported accommodation perth',
 'ndis sda',
 'disability accommodation wa',
 'ndis events perth',
 'disability service providers perth',
 'ndis short term accommodation',
 'disability housing',
 'disability homes',
 'adelphi living jobs',
 'sil housing',
 'the adelphi',
 'sda payment',
 'sda living',
 'support coordinators',
 'disability services near me',
 'adelphi house assisted living',
 'ndis wa providers',
 'ndis sda application form',
 'adelphi housing',
 'disability support organisation',
 'disability support services near me',
 'independent living supplies',
 'ndis support provider',
 'ndis',
 'ndis perth',
 'sda ndis',
 'supported independent living ndis',
 'spec

In [4]:
# Creating Class Object and assign english as language parameter
# snow_stemmer = SnowballStemmer(language='english')

# Stem every keyword from our keywords list and append them into a new list
# stem_words = []
# for keyword in keywords:
#     x = snow_stemmer.stem(keyword)
#     stem_words.append(x)

# keywords_vs_stem = pd.DataFrame({'Keywords': keywords,
#                                 'Stem Words': stem_words})
# keywords_vs_stem

# Start clustering our keywords using Machine Learning sklearn library

In [5]:
# Define our keyword clustering model with sklearn and TfidfVectorizer (using DBSCAN and not Kmean)
tfidf_vectorizer = TfidfVectorizer(max_df=0.2, max_features=10000,min_df=0.01,use_idf=True, ngram_range=(1,2))

# Import our keyword list for Keyword Clustering
tfidf_matrix = tfidf_vectorizer.fit_transform(keywords)

# Define settings for cluster adjustment (number and size of clusters)
ds = DBSCAN(eps=0.7, min_samples=5).fit(tfidf_matrix)
clusters = ds.labels_

## Create a dataframe to hold our keyword clusters

In [6]:
# Create a dataframe for our clusters
cluster_df = pd.DataFrame(clusters, columns=['Cluster Number'])

# Merge our cluster df with our top queries df
result = pd.merge(cluster_df, top_queries, left_index=True, right_index=True)

# We group our keywords by cluster number and separate them with a comma
clusters_queries = result.groupby(['Cluster Number'])['Top queries'].apply(', '.join).reset_index()

# Extend Column and Row width for a better readability 
pd.set_option('max_colwidth', 400)
pd.set_option('max_rows', 100)

clusters_queries


,Cluster Number,Top queries
0,-1,"sil providers perth, disability service providers perth, sil housing, adelphi house assisted living, adelphi housing, ndis support provider, sda accommodation, sil providers, sda house, sda house available, independent living homes, independent living services, sda perth, specialist support coordination perth, independent living solutions, housing disability perth, independent living disabled,..."
1,0,"adelphi living, adelphi, adelphi living jobs, the adelphi, freedom adelphi, adelphi hotel perth, adelphi perth, adelphi beauty, logo adelphi, adelphi group, adelphi insurance, adelphi menu, adelphi freedom, adelphi logo, 27 adelphi road claremont, adelphi springs, freedom adelphi table, air adelphi, adelphi consult, adelphi echuca, adelphi portal, adelphi ceo, clicks adelphi, adelphi hotels, a..."
2,1,"supported independent living perth, supported independent living ndis, ndis supported independent living, supported independent living, supported independent living providers, supported independent living disability, supported independent living wa, sil supported independent living, supported independent living accommodation, supported independent living ndis, supported independent living vaca..."
3,2,"ndis providers near me, disability services near me, disability support services near me, independent living near me, independent living facilities near me, ndis service providers near me, disabled living near me, disability facility near me, ndis support workers near me, disability homes near me, support workers near me, assisted accommodation near me"
4,3,"sda providers perth, ndis providers perth, sda providers victoria, sda providers, it providers perth"
5,4,"disability housing perth, ndis housing perth, disability housing, sda housing perth, disability rental housing perth, specialist disability housing perth, disability housing solutions perth, disability housing options perth, physical disability housing perth, sda housing ndis, sda housing availability, sil disability housing, disability housing wa, independent living disability housing, sda ho..."
6,5,"sil ndis, sil, sil accommodation, sil accomodation, sil sda, sil assessment ndis, sil eligibility, sil perth, sil supports, sil logo, sil living, ndia sil, sil ltd, sda sil, sil ndia, sil disability, sil insurance, sil., sil sda ndis, disability sil, www.sil, supports sil"
7,6,"disability supported accommodation perth, supported accommodation perth, supported accommodation ndis, supported accommodation for people with disabilities, ndis supported accommodation, supported accommodation, mental health supported accommodation perth, disability supported accommodation, shared supported accommodation disability services, shared supported accommodation"
8,7,"ndis sda, ndis sda application form, ndis sda properties perth, ndis sda rules, ndis sda eligibility, ndis sda funding, ndis sda fact sheet"
9,8,"ndis events perth, ndis perth, ndis mental health perth, ndis properties perth, retirement living perth, perth living, community living perth, ndis renovations perth, living perth, perth exhibitions, perth serviced apartment deal, connection perth, exhibitions perth, ndia perth, 6107 perth, simple approach perth"


## Use the same ML model on each cluster so we can name our clusters automatically

In [17]:
# Create an empty list to store our cluster names
cluster_name = []

# Use Iterrows and tfIdVectorizer to get all words and their TF-IDF Score per cluster
for index, row in clusters_queries.iterrows():
    
    queries = row['Top queries']
    queries = [queries]

    # Add a stop_words function to drop irrelevant words (ex: “a”, “the”, “is”, “are”, "and" etc...)
    my_stop_words = text.ENGLISH_STOP_WORDS

    # Run tfIdfVectorizer for each cluster
    tfIdfVectorizer=TfidfVectorizer(use_idf=True, stop_words=my_stop_words)
    tfIdf = tfIdfVectorizer.fit_transform(queries)
    query_score_cluster = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF Score"])
    query_score_cluster = query_score_cluster.sort_values('TF-IDF Score', ascending=False)

    # Get the top 3 queries (selected by highest scores) per cluster and get them into lists
    top_three_queries = query_score_cluster.index[:3]
    top_three_queries_list = top_three_queries.values.tolist()
    top_three_queries_list = str(top_three_queries_list).replace(',', '')
    top_three_queries_list = str(top_three_queries_list).replace('[','').replace(']','')

    # Append our top 3 queries to our cluster_name list
    cluster_name.append(top_three_queries_list)
    
    # Create a Cluster Name dataframe    
    cluster_name_df = pd.DataFrame({'Cluster Name': cluster_name})

cluster_name_df


,Cluster Name
0,'living' 'perth' 'ndis'
1,'adelphi' 'freedom' 'perth'
2,'independent' 'living' 'supported'
3,'near' 'disability' 'living'
4,'providers' 'perth' 'sda'
5,'housing' 'disability' 'perth'
6,'sil' 'ndis' 'sda'
7,'accommodation' 'supported' 'disability'
8,'ndis' 'sda' 'application'
9,'perth' 'ndis' 'living'


# Merge our dataframes so we can see all of our keywords grouped into different clusters and their names automatically generated

In [8]:
# Merge cluster_name_df with clusters_queries so we can have them matching and double check the cluster names match with our queries
clusters = pd.merge(cluster_name_df, clusters_queries, left_index=True, right_index=True)
clusters

,Cluster Name,Cluster Number,Top queries
0,'living' 'perth' 'ndis',-1,"sil providers perth, disability service providers perth, sil housing, adelphi house assisted living, adelphi housing, ndis support provider, sda accommodation, sil providers, sda house, sda house available, independent living homes, independent living services, sda perth, specialist support coordination perth, independent living solutions, housing disability perth, independent living disabled,..."
1,'adelphi' 'freedom' 'perth',0,"adelphi living, adelphi, adelphi living jobs, the adelphi, freedom adelphi, adelphi hotel perth, adelphi perth, adelphi beauty, logo adelphi, adelphi group, adelphi insurance, adelphi menu, adelphi freedom, adelphi logo, 27 adelphi road claremont, adelphi springs, freedom adelphi table, air adelphi, adelphi consult, adelphi echuca, adelphi portal, adelphi ceo, clicks adelphi, adelphi hotels, a..."
2,'independent' 'living' 'supported',1,"supported independent living perth, supported independent living ndis, ndis supported independent living, supported independent living, supported independent living providers, supported independent living disability, supported independent living wa, sil supported independent living, supported independent living accommodation, supported independent living ndis, supported independent living vaca..."
3,'near' 'disability' 'living',2,"ndis providers near me, disability services near me, disability support services near me, independent living near me, independent living facilities near me, ndis service providers near me, disabled living near me, disability facility near me, ndis support workers near me, disability homes near me, support workers near me, assisted accommodation near me"
4,'providers' 'perth' 'sda',3,"sda providers perth, ndis providers perth, sda providers victoria, sda providers, it providers perth"
5,'housing' 'disability' 'perth',4,"disability housing perth, ndis housing perth, disability housing, sda housing perth, disability rental housing perth, specialist disability housing perth, disability housing solutions perth, disability housing options perth, physical disability housing perth, sda housing ndis, sda housing availability, sil disability housing, disability housing wa, independent living disability housing, sda ho..."
6,'sil' 'ndis' 'sda',5,"sil ndis, sil, sil accommodation, sil accomodation, sil sda, sil assessment ndis, sil eligibility, sil perth, sil supports, sil logo, sil living, ndia sil, sil ltd, sda sil, sil ndia, sil disability, sil insurance, sil., sil sda ndis, disability sil, www.sil, supports sil"
7,'accommodation' 'supported' 'disability',6,"disability supported accommodation perth, supported accommodation perth, supported accommodation ndis, supported accommodation for people with disabilities, ndis supported accommodation, supported accommodation, mental health supported accommodation perth, disability supported accommodation, shared supported accommodation disability services, shared supported accommodation"
8,'ndis' 'sda' 'application',7,"ndis sda, ndis sda application form, ndis sda properties perth, ndis sda rules, ndis sda eligibility, ndis sda funding, ndis sda fact sheet"
9,'perth' 'ndis' 'living',8,"ndis events perth, ndis perth, ndis mental health perth, ndis properties perth, retirement living perth, perth living, community living perth, ndis renovations perth, living perth, perth exhibitions, perth serviced apartment deal, connection perth, exhibitions perth, ndia perth, 6107 perth, simple approach perth"


## Split each query at every comma and iterrate through every row so we have our queries and their corresponding cluster names matching

In [9]:
splited_clusters = pd.concat([pd.Series(row['Cluster Name'], row['Top queries'].split(','))              
                    for _, row in clusters.iterrows()]).reset_index()

In [10]:
# Display data
splited_clusters

,index,0
0,sil providers perth,'living' 'perth' 'ndis'
1,disability service providers perth,'living' 'perth' 'ndis'
2,sil housing,'living' 'perth' 'ndis'
3,adelphi house assisted living,'living' 'perth' 'ndis'
4,adelphi housing,'living' 'perth' 'ndis'
...,...,...
717,accommodation in rockingham wa,'rockingham' 'accommodation' 'wa'
718,rockingham accommodation wa,'rockingham' 'accommodation' 'wa'
719,rockingham wa accommodation,'rockingham' 'accommodation' 'wa'
720,rockingham accommodation,'rockingham' 'accommodation' 'wa'


In [11]:
# Rename our columns
splited_clusters = splited_clusters.rename(columns={'index': 'Top queries', 0: 'Cluster Name'})
splited_clusters.reset_index
splited_clusters

,Top queries,Cluster Name
0,sil providers perth,'living' 'perth' 'ndis'
1,disability service providers perth,'living' 'perth' 'ndis'
2,sil housing,'living' 'perth' 'ndis'
3,adelphi house assisted living,'living' 'perth' 'ndis'
4,adelphi housing,'living' 'perth' 'ndis'
...,...,...
717,accommodation in rockingham wa,'rockingham' 'accommodation' 'wa'
718,rockingham accommodation wa,'rockingham' 'accommodation' 'wa'
719,rockingham wa accommodation,'rockingham' 'accommodation' 'wa'
720,rockingham accommodation,'rockingham' 'accommodation' 'wa'


In [12]:
# test = []

# for index, row in splited_clusters.iterrows():
# #     for word in keywords:
#     if word in row['Top queries']:
#         test.append(df['CTR'])
# #     break
# test

NameError: name 'word' is not defined

In [ ]:
# test = []

# for index, row in clusters.iterrows():
# #     for word in keywords:
#     if word in row['Top queries']:
#         test.append(clusters['Cluster Name'])
#     break
# test

# Merge our two columns with our first Google Search Console dataframe to build our final dataframe

### We now have a complete dataframe with our queries, performances and corresponding cluster names. 

### It means we can now perform data analysis per cluster, which can be useful especially if you have a lot of queries



In [ ]:
# test_df = pd.merge(splited_clusters, df, how='outer')
# test_df = pd.merge(splited_clusters, df, left_index=True, on='Top queries', right_index=True)
# test_df

In [13]:
df

,Top queries,Clicks,Impressions,CTR,Position
0,adelphi living,289,834,34.65%,1.07
1,adelphi,15,298,5.03%,9.97
2,sil providers perth,12,90,13.33%,3.26
3,supported independent living perth,7,323,2.17%,10.77
4,adelphi living mandurah,7,30,23.33%,1.03
...,...,...,...,...,...
713,disabled living manchester,0,1,0%,134.00
714,disability insurance perth,0,1,0%,141.00
715,specialist disability housing,0,1,0%,149.00
716,disability support providers,0,1,0%,199.00


In [14]:
splited_clusters

,Top queries,Cluster Name
0,sil providers perth,'living' 'perth' 'ndis'
1,disability service providers perth,'living' 'perth' 'ndis'
2,sil housing,'living' 'perth' 'ndis'
3,adelphi house assisted living,'living' 'perth' 'ndis'
4,adelphi housing,'living' 'perth' 'ndis'
...,...,...
717,accommodation in rockingham wa,'rockingham' 'accommodation' 'wa'
718,rockingham accommodation wa,'rockingham' 'accommodation' 'wa'
719,rockingham wa accommodation,'rockingham' 'accommodation' 'wa'
720,rockingham accommodation,'rockingham' 'accommodation' 'wa'


In [16]:
# Merging our dataframes
final_df = pd.merge(df, splited_clusters, how='right', on='Top queries')
final_df.tail(20)


,Top queries,Clicks,Impressions,CTR,Position,Cluster Name
702,ndis accommodation provider,NaN,NaN,NaN,NaN,'provider' 'accommodation' 'ndia'
703,it provider perth,NaN,NaN,NaN,NaN,'provider' 'accommodation' 'ndia'
704,it provider,NaN,NaN,NaN,NaN,'provider' 'accommodation' 'ndia'
705,ndia provider,NaN,NaN,NaN,NaN,'provider' 'accommodation' 'ndia'
706,sil houses,NaN,NaN,NaN,NaN,'houses' 'living' 'ndis'
707,ndis houses,NaN,NaN,NaN,NaN,'houses' 'living' 'ndis'
708,living houses,NaN,NaN,NaN,NaN,'houses' 'living' 'ndis'
709,"""houses""",NaN,NaN,NaN,NaN,'houses' 'living' 'ndis'
710,living in wa,NaN,NaN,NaN,NaN,'living' 'perth' 'baldivis'
711,live in treeby,NaN,NaN,NaN,NaN,'living' 'perth' 'baldivis'


In [ ]:
final_df.drop(columns=['Clicks', 'Impressions', 'CTR', 'Position'])

In [ ]:
clicks_list = df['Clicks']
impressions_list = df['Impressions']
ctr_list = df['CTR']
position_list = df['Position']


In [ ]:
performance_df = pd.DataFrame({"Clicks": clicks_list,
                               "Impressions": impressions_list,
                               "CTR": ctr_list,
                               "Position": position_list})       
performance_df

In [ ]:
complete_df = pd.merge(final_df, performance_df, how='left')
complete_df

## Export our dataframe to a Google Spreadsheet (previously created) so it can be read dynamically with Tableau and Google Data Studio

In [ ]:
# Use of gspread to connect our Google Spreadsheet to our final dataframe
scope = ['https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/drive']
creds = Credentials.from_service_account_file("default.json", scopes=scope)
client = gspread.authorize(creds)
google_sh = client.open("keyword_clustering")
sheet1 = google_sh.get_worksheet(0)
sheet1.clear()
df_google = set_with_dataframe(sheet1, final_df)

In [ ]:
# Read our Google Spreadsheet to confirm the data has been correctly exported
df_google = get_as_dataframe(sheet1)
df_google.head(20)